In [1]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd
from functools import reduce
import os
import time
import numpy as np

def verificar_monitoramento(row):   
    if row['Modelo'] == 'TcA':
        return 'OK'    
    
    if row['Monitoramento: Temperatura'] == 'True' and row['Monitoramento: Aceleração'] == 'True' and row['Monitoramento: Velocidade'] == 'True':    
        return 'OK'
    else:
        return 'NOK'

def verificar_alertaS(row):
    if row['Modelo'] == 'TcAg':
        return 'OK'
        
    if row['Alerta Espectral 1: Nome'] != '' and row['Alerta Espectral 2: Nome'] != '' and row['Alerta Espectral 3: Nome'] != '':       
        return 'OK'
    else:
        return 'NOK'

def verificar_alertaT(row):
    if row['Modelo'] == 'TcA':
        if row['Alerta SW - Opção A: Ativo'] == 'Ativo' and row['Alerta SW - Opção B: Ativo'] == 'Ativo':
            return 'OK'
        else:
            return 'NOK'
        
    if row['Alerta SW - Opção A: Ativo'] == 'Ativo' and row['Alerta SW - Opção B: Ativo'] == 'Ativo' and row['Alerta SW - Opção C: Ativo'] == 'Ativo':
        return 'OK'
    else:
        return 'NOK'

def verificar_Pref(row):
    if row['Modelo'] == 'TcAg':
        return 'OK'
    
    if row['Preferência de visualização 1: Parâmetros'] != '' and row['Preferência de visualização 2: Parâmetros'] != '' and row['Preferência de visualização 3: Parâmetros'] != '':
        return 'OK'
    else:
        return 'NOK'

def verificar_RPM(row):

    if row['Modelo'] == 'TcAg':
        return 'OK'
        
    if row['RPM'] <= 3:
        return 'NOK'
    else:
        return 'OK'

def verificar_Rol(row):
    if row['Rolamento 1: Modelo'] == '' and row['Modelo'] != 'TcAg':
        return 'NOK'
    else:
        return 'OK'

def verificar_OPL(row):
    if row['Nível de Operação: Ativo'] == 'Ativo':
        return 'OK'
    else:
        return 'NOK'

def verificar_Spectral(row):

    if row['Modelo'] == 'TcAg':
        return 'OK'
    
    if row['Modelo'] == 'HF+':
        if row['RPM'] < 40 and row['Duração'] > 4.99:
            return 'OK'
        elif row['RPM'] >= 40 and row['RPM'] < 300 and row['Fmax'] <= 6572 and row['Duração'] >= 4.99 and row['Eixo'] == 'Todos':
            return 'OK'
        elif row['RPM'] >= 300 and row['RPM'] < 600 and row['Fmax'] >= 6572 and row['Duração'] >= 2.49 and row['Eixo'] == 'Todos':
            return 'OK'
        elif row['RPM'] >= 600 and row['Fmax'] >= 6572 and row['Duração'] >= 1.25 and row['Eixo'] == 'Todos':
            return 'OK'
        else:
            return 'NOK'   
            
    if row['Modelo'] == 'HF':
        if row['RPM'] < 50 and row['Duração'] > 10:
            return 'OK'
        elif row['RPM'] >= 50 and row['RPM'] < 200 and row['Fmax'] <= 3200 and row['Duração'] > 5:
            return 'OK'
        elif row['RPM'] >= 200 and row['RPM'] < 500 and row['Fmax'] >= 3200 and row['Duração'] >= 2.56 and row['Eixo'] == 'Todos':
            return 'OK'
        elif row['RPM'] >= 500 and row['RPM'] < 1000 and row['Fmax'] >= 3200 and row['Duração'] >= 1.28 and row['Eixo'] == 'Todos':
            return 'OK'
        elif row['RPM'] >= 1000 and row['Fmax'] >= 6400 and row['Duração'] >= 0.64:
            return 'OK'
        else:
            return 'NOK' 
         
    if row['Modelo'] == 'TcAs':
        if row['RPM'] <= 50 and row['Duração'] >= 12.06:
            return 'OK'
        elif row['RPM'] > 50 and row['RPM'] <= 500 and row['Fmax'] >= 2520 and row['Duração'] >= 6.05 and row['Eixo'] == 'Todos':
            return 'OK'
        elif row['RPM'] > 500 and row['Fmax'] >= 2520 and row['Duração'] >= 1.63 and row['Eixo'] == 'Todos':
            return 'OK'
        else:
            return 'NOK'   
            
    if row['Modelo'] == 'TcA+' or row['Modelo'] == 'TcA' or row['Modelo'] == 'AS':
        if row['Espectral Padrão: Eixo | Frequência | Duração'] == 'Todos | 1024 Hz | 4.00 s':
            return 'OK'
    else:
            return 'NOK' 
    


def general_status(row):
    if row['Monitoramento_Status'] == 'OK' and \
       row['AlertT_status'] == 'OK' and \
       row['AlertS_status'] == 'OK' and \
       row['RPM_status'] == 'OK' and \
       row['Pref_status'] == 'OK' and \
       row['Rolamento_status'] == 'OK' and \
       row['NivelOP_status'] == 'OK' and \
       row['Spectral_status'] == 'OK':
        return 'OK'
    else:
        return 'NOK'

###--------------------------------------------------------------------------------------------------------------------------------------


tempo_inicial = time.time()

# Cria uma janela tkinter
window = tk.Tk()
window.wm_attributes('-topmost', 1)
window.withdraw()   # this supress the tk window

# Abre a caixa de diálogo para selecionar o arquivo Excel
file_path = filedialog.askopenfilename(parent=window,filetypes=[("Excel files", "*.xlsx;*.xls")])


# Verifica se o usuário selecionou um arquivo
if file_path:

    xls = pd.ExcelFile(file_path)
    pastas = xls.sheet_names
    dfs = []
    aux = 0
    for pasta in pastas:
        df = pd.read_excel(xls, pasta)
        df = df.astype(str)
        dfs.append(df)
    result = reduce(lambda left, right: pd.merge(left, right, on=None, how='outer'), dfs)

    diretorio = os.path.dirname(file_path) + '/RESULTADO_UNIDO.xlsx'
    result.to_excel(diretorio) 
        
else:
    print("Nenhum arquivo selecionado.")


headers = ['Máquina', 'Subconjunto', 'Componente', 'spotId', 'Nome do Spot',
       'Identificador', 'Modelo', 'Monitoramento: Temperatura',
       'Monitoramento: Aceleração', 'Monitoramento: Velocidade', 'RPM',
       'Faixa Dinâmica (g)', 'Espectral Padrão: Eixo | Frequência | Duração',
       'Alerta SW - Opção A: Ativo', 'Alerta SW - Opção B: Ativo',
       'Alerta SW - Opção C: Ativo', 'Nível de Operação: Ativo',
       'Alerta Espectral 1: Nome', 'Alerta Espectral 2: Nome',
       'Alerta Espectral 3: Nome', 'Preferência de visualização 1: Parâmetros',
       'Preferência de visualização 2: Parâmetros',
       'Preferência de visualização 3: Parâmetros', 'Rolamento 1: Modelo']

for col in headers:
    if col not in result.columns:
        result[col] = ''  # Adiciona a coluna com valores vazios

dfx = result[headers].copy()
dfx[['Eixo', 'Fmax', 'Duração']] = dfx['Espectral Padrão: Eixo | Frequência | Duração'].str.split('|', expand=True)
dfx['Eixo'] = dfx['Eixo'].str.strip()
dfx['Fmax'] = dfx['Fmax'].str.strip()
dfx['Duração'] = dfx['Duração'].str.strip()
dfx['Fmax'] = pd.to_numeric(dfx['Fmax'].str.replace(' Hz', ''))
dfx['Duração'] = pd.to_numeric(dfx['Duração'].str.replace(' s', '').replace(',', '.'))

colunas_para_converter = ['RPM', 'Faixa Dinâmica (g)', 'Fmax','Duração']
for coluna in colunas_para_converter:
    dfx[coluna] = pd.to_numeric(dfx[coluna], errors='coerce')


dfx = dfx.replace('nan','')
dfx = dfx.replace(' |  | ','')
colunas_para_converter = ['Monitoramento: Temperatura','Monitoramento: Aceleração', 'Monitoramento: Velocidade']
for coluna in colunas_para_converter:
    dfx[coluna] = dfx[coluna].replace('1.0','True')
    dfx[coluna] = dfx[coluna].replace('0.0','False')

dfx = dfx.drop(dfx[dfx['Identificador'] == ''].index)

dfx = dfx.fillna('')

# Aplicando a função em cada linha para criar a nova coluna
dfx['Monitoramento_Status'] = dfx.apply(verificar_monitoramento, axis=1)
dfx['AlertT_status'] = dfx.apply(verificar_alertaT, axis=1)
dfx['AlertS_status'] = dfx.apply(verificar_alertaS, axis=1)
dfx['RPM_status'] = dfx.apply(verificar_RPM, axis=1)
dfx['Pref_status'] = dfx.apply(verificar_Pref, axis=1)
dfx['Rolamento_status'] = dfx.apply(verificar_Rol, axis=1)
dfx['NivelOP_status'] = dfx.apply(verificar_OPL, axis=1)
dfx['Spectral_status'] = dfx.apply(verificar_Spectral, axis=1)
dfx['STATUS_GERAL'] = dfx.apply(general_status, axis=1)

diretorio = os.path.dirname(file_path) + '/SPOT_INFO_LIMPO.xlsx'
dfx.to_excel(diretorio) 

colunas_status = ['STATUS_GERAL','Monitoramento_Status','Spectral_status', 'NivelOP_status', 'Rolamento_status', 
                  'Pref_status', 'RPM_status', 'AlertS_status', 'AlertT_status']

# Contar o número de 'NOK' em cada coluna de status, agrupado por modelo
resultados_status = dfx.groupby('Modelo')[colunas_status].apply(lambda x: (x == 'NOK').sum()).reset_index()

# Adicionar uma coluna 'TOTAL' mostrando o total de cada modelo
total_modelo = dfx['Modelo'].value_counts().reset_index()
total_modelo.columns = ['Modelo', 'TOTAL']

# Juntar os resultados das contagens de status com os totais de modelo
resultados_com_total = pd.merge(total_modelo, resultados_status, on='Modelo')

diretorio = os.path.dirname(file_path) + '/RESUMO_ANALISE.xlsx'
resultados_com_total.to_excel(diretorio)


tempo_final = round((time.time()-tempo_inicial)/60,1)
print('Pastas Unidas - Arquivo salvo no diretório do Input | Tempo de execução: ', tempo_final,'min')

resultados_com_total


Pastas Unidas - Arquivo salvo no diretório do Input | Tempo de execução:  0.2 min


,Modelo,TOTAL,STATUS_GERAL,Monitoramento_Status,Spectral_status,NivelOP_status,Rolamento_status,Pref_status,RPM_status,AlertS_status,AlertT_status
0,HF,234,231,2,129,78,92,74,1,124,138
1,HF+,124,117,2,38,87,74,77,17,88,68
